In [1]:
# PROCESADO DE DATOS
import numpy as np
import pandas as pd
# VISUALIZACION
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import ijson
import json
import random
# UNDER SAMPLING
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from collections import Counter

import missingno as msno
from sklearn.decomposition import PCA

inline_rc = dict(mpl.rcParams)

CARGAMOS LOS DATOS DE yelp_academic_dataset_review.json

In [2]:
reviews = []
with open('data/yelp_academic_dataset_review.json') as fl:
    i=0
    for review in fl:
        reviews.append(json.loads(review))
        i+=1
        if i + 1 > 10000:
            break

df_review = pd.DataFrame(reviews)
print(df_review.head())


                review_id                 user_id             business_id  \
0  KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   
1  BiTunyQ73aT9WBnpR9DZGw  OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ   
2  saUsX_uimxRlCVr67Z4Jig  8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A   
3  AqPFMleE6RsU23_auESxiA  _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA   
4  Sx8TMOWLNuJBWer-0pcmoA  bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ   

   stars  useful  funny  cool  \
0    3.0       0      0     0   
1    5.0       1      0     1   
2    3.0       0      0     0   
3    5.0       1      0     1   
4    4.0       1      0     1   

                                                text                 date  
0  If you decide to eat here, just be aware it is...  2018-07-07 22:09:11  
1  I've taken a lot of spin classes over the year...  2012-01-03 15:28:18  
2  Family diner. Had the buffet. Eclectic assortm...  2014-02-05 20:30:30  
3  Wow!  Yummy, different,  delici

CARGAMOS LOS DATOS DE yelp_academic_dataset_business.json

In [3]:
business = []
with open('data/yelp_academic_dataset_business.json') as fl:
    for i, line in enumerate(fl):
        business.append(json.loads(line))

df_business = pd.DataFrame(business)
print(df_business.head())



              business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address           city state postal_code  \
0           1616 Chapala St, Ste 2  Santa Barbara    CA       93101   
1  87 Grasso Plaza Shopping Center         Affton    MO       63123   
2             5255 E Broadway Blvd         Tucson    AZ       85711   
3                      935 Race St   Philadelphia    PA       19107   
4                    101 Walnut St     Green Lane    PA       18054   

    latitude   longitude  stars  review_count  is_open  \
0  34.426679 -119.711197    5.0             7        0   
1  38.551126  -90.335695    3.0            15        1   
2  32.223236 -110.880452    3.5            22        0   
3  39.9555

In [4]:
useful_reviews = len(df_review[df_review["useful"]>0])
cool_reviews = len(df_review[df_review["cool"]>0])
funny_reviews = len(df_review[df_review["funny"]>0])
negative_reviws = len(df_review[df_review["stars"]<2])
positive_reviews =len(df_review[df_review["stars"]>3])
total_reviews = len(df_review)

print("Total reviews: {}".format(total_reviews))
print("Useful reviews: {}".format(useful_reviews))
print("Funny reviews: {}".format(funny_reviews))
print("Cool reviews: {}".format(cool_reviews))
print("Total negative reviews: {}".format(negative_reviws))
print("Total positive reviews: {}".format(positive_reviews))

Total reviews: 10000
Useful reviews: 4001
Funny reviews: 1377
Cool reviews: 1964
Total negative reviews: 1079
Total positive reviews: 7019


ELIMINAMOS LAS COLUMNAS INNECESARIAS

In [4]:
df_review = df_review.drop(["review_id", "user_id", "useful", "funny", "cool", "date"], axis=1)
df_business = df_business.drop(["name", "address", "city", "state", "postal_code", "latitude", "longitude", "stars", "review_count", "is_open", "attributes", "hours"], axis=1)

COMPROBAMOS

In [5]:
print(df_review.head())
print(df_business.head())

              business_id  stars  \
0  XQfwVwDr-v0ZS3_CbbE5Xw    3.0   
1  7ATYjTIgM3jUlt4UM3IypQ    5.0   
2  YjUWPpI6HXG530lwP-fb2A    3.0   
3  kxX2SOes4o-D3ZQBkiMRfA    5.0   
4  e4Vwtrqf-wpJfwesgvdgxQ    4.0   

                                                text  
0  If you decide to eat here, just be aware it is...  
1  I've taken a lot of spin classes over the year...  
2  Family diner. Had the buffet. Eclectic assortm...  
3  Wow!  Yummy, different,  delicious.   Our favo...  
4  Cute interior and owner (?) gave us tour of up...  
              business_id                                         categories
0  Pns2l4eNsfO8kk83dixA6A  Doctors, Traditional Chinese Medicine, Naturop...
1  mpf3x-BjTdTEA3yCZrAYPw  Shipping Centers, Local Services, Notaries, Ma...
2  tUFrWirKiKi_TAnsVWINQQ  Department Stores, Shopping, Fashion, Home & G...
3  MTSW4McQd7CbVtyjqoe9mw  Restaurants, Food, Bubble Tea, Coffee & Tea, B...
4  mWMc6_wTdE0EUBKIGXDVfA                          Brewpubs, Breweri

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

vectorizer = TfidfVectorizer()
lemma = WordNetLemmatizer()

X = []
for r in df_review["text"]: #Por cada review
    txt = word_tokenize(r.lower()) #Separa la review en palabras
    newtxt = ""
    for w in txt: #Por cada palabra en txt
        l = lemma.lemmatize(w) #se hace lo de quitar mayusculas y quitar raice y eso
        newtxt += l+" "
    X.append(newtxt)

X = vectorizer.fit_transform(X)
print(X)

  (0, 15619)	0.2104458399617418
  (0, 6192)	0.10855609151571549
  (0, 13782)	0.08076099116709685
  (0, 21438)	0.1180594895609946
  (0, 13680)	0.04859720545927605
  (0, 16371)	0.07744551748037877
  (0, 13771)	0.068812982404813
  (0, 5883)	0.14191191314453558
  (0, 1239)	0.09584494993848772
  (0, 7891)	0.039774201657384746
  (0, 13758)	0.1978799667774664
  (0, 21251)	0.11570892814183979
  (0, 21397)	0.08697137457813488
  (0, 18397)	0.13848451095725975
  (0, 21524)	0.09891755558593203
  (0, 11983)	0.09493405531570533
  (0, 19973)	0.15421240249770168
  (0, 14856)	0.1284627305556453
  (0, 20843)	0.22486066501780297
  (0, 21976)	0.14533526178170164
  (0, 21257)	0.17080829503104247
  (0, 13861)	0.06155823009612091
  (0, 4368)	0.08012189900768543
  (0, 11620)	0.19381910108012493
  (0, 21027)	0.11652890245654866
  :	:
  (9999, 15838)	0.1276773925983044
  (9999, 21625)	0.06196581503787413
  (9999, 1429)	0.08000467750412457
  (9999, 8581)	0.05835544214372858
  (9999, 14793)	0.046034746199690106
 

In [7]:
X= X.toarray()
X=pd.DataFrame(X)
print(X)

      0      1      2      3      4      5      6      7      8      9      \
0       0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1       0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2       0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3       0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4       0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
9995    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
9996    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
9997    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
9998    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
9999    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

      ...  22107  22108  22109  22110  22111  22112  22113  221

In [8]:
X.describe()

KeyboardInterrupt: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Seaborn Plot Styling
sns.set(style="white", palette="husl")
sns.set_context("poster")
sns.set_style("ticks")

%matplotlib inline

# New imports, specific to this example
from scipy.cluster.hierarchy import dendrogram, linkage

def mai():
  D = 2                     # Dimensionality
  s = 4                     # Separation so we can control how far apart the means are
  mu1 = np.array([0, 0])
  mu2 = np.array([s, s])
  mu3 = np.array([0, s])

  # Call linkage on X, pass in parameter telling what type of linkage to use
  Z = linkage(X, 'ward')
  # In general, Z will have format index1, index2 for two index's in X that represent the
  # points that are joined at that moment. The third column will be the distance, which is
  # how far apart the two clusters were before being joined, and the 4th column will be
  # sample count, which is the number of points in that cluster. Hence, the size of Z will
  # be ((N - 1) x 4)
  print("Z shape", Z.shape)

  fig, ax = plt.subplots(figsize=(12,8))
  plt.title("Ward")
  dendrogram(Z)      # Call dendrogram on Z
  plt.show()

  # Now do single linkage for a different example
  fig, ax = plt.subplots(figsize=(12,8))
  Z = linkage(X, 'single')
  plt.title("Single")
  dendrogram(Z)
  plt.show()

  # And one more for complete
  fig, ax = plt.subplots(figsize=(12,8))
  Z = linkage(X, 'complete')
  plt.title("Complete")
  dendrogram(Z)
  plt.show()


In [ ]:
mai()

In [ ]:
class Distance_computation_grid(object):
    '''
        class to enable the Computation of distance matrix
    '''
    def __init__(self):
        pass

    def compute_distance(self,samples):
        '''
            Creates a matrix of distances between individual samples and clusters attained at a particular step
        '''
        Distance_mat = np.zeros((len(samples),len(samples)))
        for i in range(Distance_mat.shape[0]):
            for j in range(Distance_mat.shape[0]):
                if i!=j:
                    Distance_mat[i,j] = float(self.distance_calculate(samples[i],samples[j]))
                else:
                    Distance_mat[i,j] = 10**4
        return Distance_mat


    def distance_calculate(self,sample1,sample2):
        '''
            Distance calulated between two samples. The two samples can be both samples, both clusters or
            one cluster and one sample. If both of them are samples/clusters, then simple norm is used. In other
            cases, we refer it as an exception case and pass the samples as parameter to some function that
            calculates the necessary distance between cluster and a sample
        '''
        dist = []
        for i in range(len(sample1)):
            for j in range(len(sample2)):
                try:
                    dist.append(np.linalg.norm(np.array(sample1[i])-np.array(sample2[j])))
                except:
                    dist.append(self.intersampledist(sample1[i],sample2[j]))
        return min(dist)


    def intersampledist(self,s1,s2):
        '''
            To be used in case we have one sample and one cluster . It takes the help of one
            method 'interclusterdist' to compute the distances between elements of a cluster(which are
            samples) and the actual sample given.
        '''
        if str(type(s2[0]))!='<class \'list\'>':
            s2=[s2]
        if str(type(s1[0]))!='<class \'list\'>':
            s1=[s1]
        m = len(s1)
        n = len(s2)
        dist = []
        if n>=m:
            for i in range(n):
                for j in range(m):
                    if (len(s2[i])>=len(s1[j])) and str(type(s2[i][0])!='<class \'list\'>'):
                        dist.append(self.interclusterdist(s2[i],s1[j]))
                    else:
                        dist.append(np.linalg.norm(np.array(s2[i])-np.array(s1[j])))
        else:
            for i in range(m):
                for j in range(n):
                    if (len(s1[i])>=len(s2[j])) and str(type(s1[i][0])!='<class \'list\'>'):
                        dist.append(self.interclusterdist(s1[i],s2[j]))
                    else:
                        dist.append(np.linalg.norm(np.array(s1[i])-np.array(s2[j])))
        return min(dist)

    def interclusterdist(self,cl,sample):
        if sample[0]!='<class \'list\'>':
            sample = [sample]
        dist   = []
        for i in range(len(cl)):
            for j in range(len(sample)):
                dist.append(np.linalg.norm(np.array(cl[i])-np.array(sample[j])))
        return min(dist)

In [ ]:
progression = [[i] for i in range(X.shape[0])]
samples     = [[list(X[i])] for i in range(X.shape[0])]
m = len(samples)
distcal  = Distance_computation_grid()

while m>1:
    print('Sample size before clustering    :- ',m)
    Distance_mat      = distcal.compute_distance(samples)
    sample_ind_needed = np.where(Distance_mat==Distance_mat.min())[0]
    value_to_add      = samples.pop(sample_ind_needed[1])
    samples[sample_ind_needed[0]].append(value_to_add)

    print('Cluster Node 1                   :-',progression[sample_ind_needed[0]])
    print('Cluster Node 2                   :-',progression[sample_ind_needed[1]])

    progression[sample_ind_needed[0]].append(progression[sample_ind_needed[1]])
    progression[sample_ind_needed[0]] = [progression[sample_ind_needed[0]]]
    v = progression.pop(sample_ind_needed[1])
    m = len(samples)

    print('Progression(Current Sample)      :-',progression)
    print('Cluster attained                 :-',progression[sample_ind_needed[0]])
    print('Sample size after clustering     :-',m)
    print('\n')

In [ ]:
# Import the fcluster and linkage functions
from scipy.cluster.hierarchy import fcluster, linkage

# Use the linkage() function
distance_matrix = linkage(X, method = 'ward', metric = 'euclidean')

In [ ]:
# Import the dendrogram function
from scipy.cluster.hierarchy import dendrogram

# Create a dendrogram
dn = dendrogram(distance_matrix)

# Display the dendogram
plt.show()